In [1]:
# initial setup
#%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# LAB: Estimando hiperparámetros con `GridSearchCV` para Regresión Logística y KNN

## Introducción

El objetivo de esta práctica es que puedan comenzar a tunear hiperparámetros usando Cross Validation. Para eso, usaremos `GridSearchCV`.

Utilizaremos un dataset sobre cáncer de mama. Contiene información de estudios clínicos y celulares. El objetivo es predecir el carácter benigno ($class_t=0$) maligno ($class_t=1$) del cáncer en función de una serie de predictores a nivel celular.

    + class_t es la variable target
    + el resto son variables con valores normalizados de 1 a 10

[Aquí](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names) pueden encontrar más información sobre el dataset.

**Nota:** se eliminaron del dataset original 16 casos con valores perdidos en algunos campos.

## Tareas

Para esta práctica deberá 

1. Construir dos clasificadores: Regresión Logística y K-Vecinos más Cercanos (KNN)
2. Estimar los hiperpáremetros del modelo
    
    - 2.1 **LogReg:** deberá tunear un modelo con solver 'saga', C's = 1, 10, 100, 1000, y regularización L1 y L2
    - 2.2 **KNN:** deberá tunear tanto el parámetro k, como la medida del peso dado a los K vecinos (uniforme o distancia). También podría probar con el parámetro p que define el tipo de distancia con el que se calculan los vecinos más cercanos.
      
      
3. Estimar los modelos finales
4. Evaluar cuál de los dos performa mejor

**Importante:** recuerde que deberá diseñar cuidadosamente las diferentes estrategias de validación de las diferentes etapas de estimación del modelo.

Importamos los paquetes necesarios


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import datasets
from sklearn.preprocessing import normalize

Importamos el dataset

In [3]:
df = pd.read_csv('../Data/breast-cancer.csv', header = None)
df.columns = ['ID', 'clump_Thickness', 'unif_cell_size', 'unif_cell_shape', 'adhesion', 'epith_cell_Size', 'bare_nuclei',
              'bland_chromatin ','norm_nucleoli', 'mitoses', 'class_t']

In [4]:
df.shape

(683, 11)

In [5]:
df.head()

,ID,clump_Thickness,unif_cell_size,unif_cell_shape,adhesion,epith_cell_Size,bare_nuclei,bland_chromatin,norm_nucleoli,mitoses,class_t
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


Recodificamos las clases en "0" y "1"

In [6]:
df.class_t[df['class_t'] == 2] = 0
df.class_t[df['class_t'] == 4] = 1

In [7]:
df['class_t'].value_counts(normalize=True)

0    0.650073
1    0.349927
Name: class_t, dtype: float64

Hacemos el split entre target y features

In [8]:
X = df.iloc[:,1:9]
y = df['class_t']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,stratify=df['class_t'])

¿Hace falta estandarizar en este caso?

In [9]:
# Utilizamos sklearn para estandarizar la matriz de Features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

**Pista 1:** Conviene realizar dos listas, una con los estimadores de los modelos y otra con la grid de parámetros a estimar en cada modelo.

**Pista 2:** Conviene iterar sobre esas listas para estimar los hiperparámetros de los modelos

In [10]:
models = [LogisticRegression(),
          KNeighborsClassifier()]

In [11]:
params = [
    {'C': [1, 10, 100, 1000],
     'penalty': ['l1', 'l2',],
     'solver': ['saga']},
    {'n_neighbors': range(1,200),
     'weights' : ['uniform', 'distance'],
     'p' : [1, 2, 3]}
]

In [12]:
from sklearn.model_selection import StratifiedKFold
folds=StratifiedKFold(n_splits=10, random_state=19, shuffle=True)

In [13]:
grids = []
for i in range(len(models)):
    gs = GridSearchCV(estimator=models[i], param_grid=params[i], scoring='accuracy', cv=folds, n_jobs=4)
    print (gs)
    fit = gs.fit(X_train, y_train)
    grids.append(fit)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=19, shuffle=True),
             estimator=LogisticRegression(), n_jobs=4,
             param_grid={'C': [1, 10, 100, 1000], 'penalty': ['l1', 'l2'],
                         'solver': ['saga']},
             scoring='accuracy')
GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=19, shuffle=True),
             estimator=KNeighborsClassifier(), n_jobs=4,
             param_grid={'n_neighbors': range(1, 200), 'p': [1, 2, 3],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')


In [14]:
for i in grids:
    print (i.best_score_)
    print (i.best_estimator_)
    print (i.best_params_)

0.9534756097560976
LogisticRegression(C=10, penalty='l1', solver='saga')
{'C': 10, 'penalty': 'l1', 'solver': 'saga'}
0.965731707317073
KNeighborsClassifier(n_neighbors=12, p=1, weights='distance')
{'n_neighbors': 12, 'p': 1, 'weights': 'distance'}


0.965731707317073

LogisticRegression(C=1, solver='saga')

{'C': 1, 'penalty': 'l2', 'solver': 'saga'}

0.9779268292682926

KNeighborsClassifier(n_neighbors=6, p=3, weights='distance')

{'n_neighbors': 6, 'p': 3, 'weights': 'distance'}

In [15]:
pd.DataFrame(grids[0].cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,param_solver,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009375,0.001018,0.002094,0.000299,1,l1,saga,"{'C': 1, 'penalty': 'l1', 'solver': 'saga'}",0.97561,1.0,...,0.97561,0.926829,0.878049,0.97561,1.00000,1.0,0.925,0.965671,0.039813,2
1,0.010672,0.002445,0.001695,0.000457,1,l2,saga,"{'C': 1, 'penalty': 'l2', 'solver': 'saga'}",0.97561,1.0,...,0.97561,0.926829,0.853659,0.97561,1.00000,1.0,0.950,0.965732,0.043945,1
2,0.012965,0.004348,0.001895,0.000299,10,l1,saga,"{'C': 10, 'penalty': 'l1', 'solver': 'saga'}",0.97561,1.0,...,0.97561,0.926829,0.853659,0.97561,0.97561,1.0,0.950,0.963293,0.042632,3
3,0.007978,0.001261,0.001795,0.000399,10,l2,saga,"{'C': 10, 'penalty': 'l2', 'solver': 'saga'}",0.97561,1.0,...,0.97561,0.926829,0.853659,0.97561,0.97561,1.0,0.950,0.963293,0.042632,3
4,0.008278,0.000457,0.001695,0.000457,100,l1,saga,"{'C': 100, 'penalty': 'l1', 'solver': 'saga'}",0.97561,1.0,...,0.97561,0.926829,0.853659,0.97561,0.97561,1.0,0.950,0.963293,0.042632,3
5,0.007081,0.000537,0.001396,0.000488,100,l2,saga,"{'C': 100, 'penalty': 'l2', 'solver': 'saga'}",0.97561,1.0,...,0.97561,0.926829,0.853659,0.97561,0.97561,1.0,0.950,0.963293,0.042632,3
6,0.008277,0.000457,0.001496,0.000499,1000,l1,saga,"{'C': 1000, 'penalty': 'l1', 'solver': 'saga'}",0.97561,1.0,...,0.97561,0.926829,0.853659,0.97561,0.97561,1.0,0.950,0.963293,0.042632,3
7,0.006881,0.000299,0.001396,0.000489,1000,l2,saga,"{'C': 1000, 'penalty': 'l2', 'solver': 'saga'}",0.97561,1.0,...,0.97561,0.926829,0.853659,0.97561,0.97561,1.0,0.950,0.963293,0.042632,3


In [16]:
pd.DataFrame(grids[1].cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006000,0.002582,0.008478,0.001684,1,1,uniform,"{'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}",1.000000,0.975610,...,1.000000,0.975610,0.975610,1.000000,0.951220,1.000000,1.000,0.985366,0.016179,4
1,0.002393,0.000488,0.002194,0.000399,1,1,distance,"{'n_neighbors': 1, 'p': 1, 'weights': 'distance'}",1.000000,0.975610,...,1.000000,0.975610,0.975610,1.000000,0.951220,1.000000,1.000,0.985366,0.016179,4
2,0.001496,0.000498,0.002992,0.000446,1,2,uniform,"{'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}",1.000000,0.975610,...,1.000000,0.951220,0.926829,1.000000,0.951220,1.000000,1.000,0.980488,0.026269,12
3,0.001296,0.000457,0.001197,0.000399,1,2,distance,"{'n_neighbors': 1, 'p': 2, 'weights': 'distance'}",1.000000,0.975610,...,1.000000,0.951220,0.926829,1.000000,0.951220,1.000000,1.000,0.980488,0.026269,12
4,0.001097,0.000299,0.003291,0.000457,1,3,uniform,"{'n_neighbors': 1, 'p': 3, 'weights': 'uniform'}",0.975610,0.951220,...,1.000000,0.951220,0.926829,1.000000,0.926829,1.000000,1.000,0.968293,0.028962,207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,0.000296,0.000453,0.001696,0.000457,199,1,distance,"{'n_neighbors': 199, 'p': 1, 'weights': 'dista...",0.878049,0.853659,...,1.000000,0.878049,0.926829,0.902439,0.926829,0.951220,0.950,0.924268,0.044167,1044
1190,0.000599,0.000489,0.002792,0.000598,199,2,uniform,"{'n_neighbors': 199, 'p': 2, 'weights': 'unifo...",0.853659,0.878049,...,0.926829,0.878049,0.902439,0.853659,0.902439,0.926829,0.925,0.897378,0.028260,1185
1191,0.000499,0.000499,0.001496,0.000499,199,2,distance,"{'n_neighbors': 199, 'p': 2, 'weights': 'dista...",0.878049,0.878049,...,1.000000,0.902439,0.926829,0.902439,0.926829,0.951220,0.950,0.924268,0.035170,1044
1192,0.000801,0.000599,0.005790,0.000396,199,3,uniform,"{'n_neighbors': 199, 'p': 3, 'weights': 'unifo...",0.853659,0.878049,...,0.902439,0.878049,0.878049,0.853659,0.902439,0.951220,0.925,0.892500,0.029059,1193


In [17]:
X_test = scaler.transform(X_test)

In [18]:
y_preds_log = grids[0].predict(X_test)
y_preds_knn = grids[1].predict(X_test)

In [21]:
print (classification_report(y_test, y_preds_log))

              precision    recall  f1-score   support

           0       0.99      0.94      0.97       178
           1       0.90      0.99      0.95        96

    accuracy                           0.96       274
   macro avg       0.95      0.97      0.96       274
weighted avg       0.96      0.96      0.96       274



In [23]:
confusion_matrix(y_test, y_preds_log)

array([[168,  10],
       [  1,  95]], dtype=int64)

In [25]:
print (classification_report(y_test, y_preds_knn))

              precision    recall  f1-score   support

           0       0.98      0.95      0.97       178
           1       0.91      0.97      0.94        96

    accuracy                           0.96       274
   macro avg       0.95      0.96      0.95       274
weighted avg       0.96      0.96      0.96       274



In [26]:
confusion_matrix(y_test, y_preds_knn)

array([[169,   9],
       [  3,  93]], dtype=int64)

## Diferencia de performance entre Random Search y Gridsearch

Dado el siguiente conjunto de parámetros:

        param_dist = {
                    'n_neighbors': range(1,200),
                    'weights' : ['uniform', 'distance'],
                    'p' : [1, 2, 3]
                    }

Implementar una búsqueda del conjunto óptimo de hiperparámetros tanto con GridSearchCV como con RandomSearchCV.
Verificar la diferencia en cada caso de:
    
    1. El tiempo de ejecución (utilizando la biblioteca time)
    2. La combinación óptima de parámetros
    3. La performance del mejor modelo en cada caso sobre los datos del test set que separamos anteriormente en términos de accuracy


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
def busquedaGridsearch(params_):
    folds=StratifiedKFold(n_splits=10, random_state=19, shuffle=True)
    gs = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=params_, scoring='accuracy', cv=folds, n_jobs=4)
    fit = gs.fit(X_train, y_train)
    return gs    

In [ ]:
def busquedaRandomSearch(params_,iter_):
    folds=StratifiedKFold(n_splits=10, random_state=19, shuffle=True)
    gs = RandomizedSearchCV(estimator=KNeighborsClassifier(), param_distributions=params_, scoring='accuracy', cv=folds, n_jobs=4, n_iter = iter_ )
    fit = gs.fit(X_train, y_train)
    return gs    

In [ ]:
param_dist = {
    'n_neighbors': range(1,200),
    'weights' : ['uniform', 'distance'],
    'p' : [1, 2, 3]
}


In [ ]:
import time

In [ ]:
tic = time.time()
gs_random_search = busquedaRandomSearch(param_dist,100)        
toc = time.time()
print(str(toc-tic) + ' Segundos')

In [ ]:
tic = time.time()
gs_grid_search = busquedaGridsearch(param_dist)   
toc = time.time()
print(str(toc-tic) + ' Segundos')

In [ ]:
gs_random_search.best_params_

In [ ]:
gs_grid_search.best_params_

In [ ]:
from sklearn.metrics import accuracy_score

def obtener_performance(estimator):
    y_pred = estimator.predict(X_test)
    return accuracy_score(y_pred,y_test, normalize = True)

In [ ]:
obtener_performance(gs_grid_search.best_estimator_)

In [ ]:
obtener_performance(gs_random_search.best_estimator_)